<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Fake News Detection System
# A step-by-step implementation of the research methodology

# Import necessary libraries
import pandas as pd
import numpy as np
import re
import nltk
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Step 1: Dataset Loading
def load_dataset():
    """
    Load the Kaggle fake and real news dataset using kagglehub.
    """
    print("Step 1: Loading Dataset...")

    # Download the dataset
    path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
    print(f"Path to dataset files: {path}")

    # Load the data
    # This dataset has separate files for true and fake news
    true_news_path = os.path.join(path, "True.csv")
    fake_news_path = os.path.join(path, "Fake.csv")

    true_news = pd.read_csv(true_news_path)
    fake_news = pd.read_csv(fake_news_path)

    # Add labels to the datasets
    true_news['label'] = 1  # 1 for true news
    fake_news['label'] = 0  # 0 for fake news

    # Combine the datasets
    df = pd.concat([true_news, fake_news], ignore_index=True)

    # Shuffle the dataset
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Display dataset information
    print(f"Dataset loaded successfully with {len(df)} entries.")
    print(f"True news: {len(true_news)} entries")
    print(f"Fake news: {len(fake_news)} entries")
    print(f"Columns in the dataset: {df.columns.tolist()}")

    return df

# Step 2: Data Cleaning
def clean_data(df):
    """
    Clean the dataset by removing null values and unnecessary columns.
    """
    print("Step 2: Cleaning Data...")

    # Check for null values
    print(f"Null values before cleaning: {df.isnull().sum().sum()}")

    # Drop rows with null values
    df = df.dropna()

    # For this specific dataset, we'll focus on the text, title, and label columns
    if 'title' in df.columns and 'text' in df.columns:
        df = df[['title', 'text', 'label']]
    elif 'title' in df.columns and 'text' not in df.columns:
        # If 'text' is not present but 'content' is (common alternative name)
        if 'content' in df.columns:
            df = df.rename(columns={'content': 'text'})
            df = df[['title', 'text', 'label']]

    print(f"Null values after cleaning: {df.isnull().sum().sum()}")
    print(f"Dataset shape after cleaning: {df.shape}")

    return df

# Step 3: Text Processing (NLP)
def preprocess_text(df):
    """
    Perform text preprocessing using NLP techniques:
    - Tokenization
    - Stop words removal
    - Punctuation removal
    - Stemming
    """
    print("Step 3: Preprocessing Text...")

    # Initialize stemmer
    stemmer = PorterStemmer()

    # Get English stopwords
    stop_words = set(stopwords.words('english'))

    def clean_text(text):
        # Convert to lowercase
        text = str(text).lower()

        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)

        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)

        # Remove punctuation and special characters
        text = re.sub(r'[^\w\s]', '', text)

        # Remove numbers
        text = re.sub(r'\d+', '', text)

        # Tokenize
        tokens = nltk.word_tokenize(text)

        # Remove stopwords and apply stemming
        cleaned_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and len(word) > 2]

        return ' '.join(cleaned_tokens)

    # Apply text cleaning to the 'text' column
    df['processed_text'] = df['text'].apply(clean_text)

    # If title column exists, process it too and combine with text
    if 'title' in df.columns:
        df['processed_title'] = df['title'].apply(clean_text)
        df['processed_content'] = df['processed_title'] + ' ' + df['processed_text']
    else:
        df['processed_content'] = df['processed_text']

    print("Text preprocessing completed.")
    return df

# Step 4: Feature Extraction (Vectorization)
def vectorize_text(df_train, df_test, method='tfidf'):
    """
    Convert text to numerical vectors using either TF-IDF or Bag of Words
    """
    print(f"Step 4: Vectorizing Text using {method}...")

    if method == 'tfidf':
        # TF-IDF Vectorization
        vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    else:
        # Bag of Words Vectorization
        vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))

    # Fit and transform the training data
    X_train = vectorizer.fit_transform(df_train['processed_content'])

    # Transform the test data
    X_test = vectorizer.transform(df_test['processed_content'])

    print(f"Vectorization completed. Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")
    return X_train, X_test, vectorizer

# Step 5: Machine Learning Models
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    """
    Train and evaluate multiple machine learning models:
    - Random Forest
    - Decision Tree
    - Support Vector Machine
    """
    print("Step 5: Training and Evaluating Machine Learning Models...")

    # Define models
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'SVM': SVC(kernel='linear', random_state=42, probability=True)
    }

    results = {}

    for name, model in models.items():
        print(f"\nTraining {name}...")

        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)

        # Generate classification report
        report = classification_report(y_test, y_pred, output_dict=True)

        # Generate confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'report': report,
            'confusion_matrix': cm
        }

        print(f"{name} Accuracy: {accuracy:.4f}")
        print(f"{name} Classification Report:")
        print(classification_report(y_test, y_pred))

    return results

# Step 6: Visualize Results
def visualize_results(results):
    """
    Visualize the performance of different models
    """
    print("Step 6: Visualizing Results...")

    # Plot accuracy comparison
    plt.figure(figsize=(10, 6))
    accuracies = [results[model]['accuracy'] for model in results.keys()]
    plt.bar(results.keys(), accuracies, color=['blue', 'green', 'red'])
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Models')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    for i, v in enumerate(accuracies):
        plt.text(i, v + 0.01, f"{v:.4f}", ha='center')
    plt.savefig('model_accuracy_comparison.png')
    plt.close()

    # Plot confusion matrices
    for name, result in results.items():
        plt.figure(figsize=(8, 6))
        cm = result['confusion_matrix']
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Fake', 'True'],
                   yticklabels=['Fake', 'True'])
        plt.title(f'Confusion Matrix - {name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig(f'confusion_matrix_{name.replace(" ", "_").lower()}.png')
        plt.close()

    # Plot precision, recall, and F1-score
    plt.figure(figsize=(12, 6))
    model_names = list(results.keys())
    precision = [results[model]['report']['1']['precision'] for model in model_names]
    recall = [results[model]['report']['1']['recall'] for model in model_names]
    f1 = [results[model]['report']['1']['f1-score'] for model in model_names]

    x = np.arange(len(model_names))
    width = 0.25

    plt.bar(x - width, precision, width, label='Precision')
    plt.bar(x, recall, width, label='Recall')
    plt.bar(x + width, f1, width, label='F1-score')

    plt.xlabel('Models')
    plt.ylabel('Score')
    plt.title('Precision, Recall, and F1-score for True News Classification')
    plt.xticks(x, model_names)
    plt.legend()
    plt.savefig('precision_recall_f1_comparison.png')
    plt.close()

    print("Visualization completed. Results saved as images.")

# Function to extract important features
def extract_important_features(vectorizer, model, top_n=20):
    """
    Extract the most important features for classification
    """
    if hasattr(model, 'feature_importances_'):
        # For tree-based models
        feature_importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        # For linear models like SVM
        feature_importances = np.abs(model.coef_[0])
    else:
        print("Model does not support feature importance extraction")
        return None

    # Get feature names
    feature_names = vectorizer.get_feature_names_out()

    # Create a DataFrame of features and their importance
    features_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importances
    })

    # Sort by importance
    features_df = features_df.sort_values('importance', ascending=False)

    # Get top N features
    top_features = features_df.head(top_n)

    return top_features

# Main function to run the entire pipeline
def main():
    """
    Main function to execute the complete fake news detection pipeline
    """
    print("Starting Fake News Detection System...")

    # Step 1: Load dataset
    df = load_dataset()

    # Step 2: Clean data
    df = clean_data(df)

    # Step 3: Preprocess text
    df = preprocess_text(df)

    # Split data into training and testing sets (80:20 ratio)
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

    # Extract labels
    y_train = df_train['label']
    y_test = df_test['label']

    # Step 4: Vectorize text
    X_train, X_test, vectorizer = vectorize_text(df_train, df_test, method='tfidf')

    # Step 5: Train and evaluate models
    results = train_and_evaluate_models(X_train, X_test, y_train, y_test)

    # Step 6: Visualize results
    visualize_results(results)

    # Find the best model
    best_model_name = max(results, key=lambda x: results[x]['accuracy'])
    best_model = results[best_model_name]['model']
    best_accuracy = results[best_model_name]['accuracy']

    print(f"\nBest performing model: {best_model_name} with accuracy {best_accuracy:.4f}")

    # Extract important features from the best model
    if best_model_name in ['Random Forest', 'Decision Tree', 'SVM']:
        print("\nTop 20 important features for classification:")
        top_features = extract_important_features(vectorizer, best_model)
        print(top_features)

        # Visualize top features
        plt.figure(figsize=(12, 8))
        sns.barplot(x='importance', y='feature', data=top_features)
        plt.title(f'Top 20 Important Features for {best_model_name}')
        plt.tight_layout()
        plt.savefig('top_features.png')
        plt.close()

    print("\nFake News Detection System completed successfully!")

    # Save the best model and vectorizer for future use
    import joblib
    joblib.dump(best_model, f'best_model_{best_model_name.lower().replace(" ", "_")}.pkl')
    joblib.dump(vectorizer, 'vectorizer.pkl')
    print(f"Best model and vectorizer saved for future use.")

    return results, vectorizer, best_model

# Function to make predictions on new data
def predict_news(text, model, vectorizer):
    """
    Make predictions on new news articles
    """
    # Preprocess the text
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    tokens = nltk.word_tokenize(text)
    cleaned_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and len(word) > 2]
    processed_text = ' '.join(cleaned_tokens)

    # Vectorize the text
    text_vector = vectorizer.transform([processed_text])

    # Make prediction
    prediction = model.predict(text_vector)[0]
    prediction_proba = model.predict_proba(text_vector)[0]

    # Return result
    if prediction == 1:
        return "True News", prediction_proba[1]
    else:
        return "Fake News", prediction_proba[0]

# Example usage of the system
if __name__ == "__main__":
    # Run the main pipeline
    results, vectorizer, best_model = main()

    # Example news article for prediction
    sample_news = """
    Scientists discover new treatment for cancer that shows promising results in early trials.
    The research team at University Medical Center has developed a novel approach that targets specific cancer cells
    without damaging healthy tissue. The treatment has been tested on a small group of patients with advanced forms of
    cancer, and initial results show significant tumor reduction in 85% of participants. The team plans to expand the
    clinical trials next year after receiving additional funding from the National Health Institute.
    """

    # Make prediction
    result, confidence = predict_news(sample_news, best_model, vectorizer)
    print(f"\nPrediction for sample news: {result} (Confidence: {confidence:.4f})")

    # Create a simple function for users to input their own news articles
    def user_input_prediction():
        print("\n=== Fake News Detector ===")
        print("Enter a news article to check if it's real or fake (type 'exit' to quit):")

        while True:
            user_input = input("\nEnter news text: ")

            if user_input.lower() == 'exit':
                print("Exiting fake news detector.")
                break

            if len(user_input) < 50:
                print("Please enter a longer news article for more accurate prediction.")
                continue

            result, confidence = predict_news(user_input, best_model, vectorizer)
            print(f"Prediction: {result} (Confidence: {confidence:.4f})")

    # Uncomment the line below to enable interactive user input
    # user_input_prediction()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Starting Fake News Detection System...
Step 1: Loading Dataset...


100%|██████████| 41.0M/41.0M [00:00<00:00, 121MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/clmentbisaillon/fake-and-real-news-dataset/versions/1
Dataset loaded successfully with 44898 entries.
True news: 21417 entries
Fake news: 23481 entries
Columns in the dataset: ['title', 'text', 'subject', 'date', 'label']
Step 2: Cleaning Data...
Null values before cleaning: 0
Null values after cleaning: 0
Dataset shape after cleaning: (44898, 3)
Step 3: Preprocessing Text...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
